In [1]:
def calc(file):
    import pandas as pd
    import numpy as np
    from numpy.lib.stride_tricks import sliding_window_view
    from scipy.signal import stft
    from os.path import isfile
    
    def getParams (fs, alphaMax=1000, df=100, fMin=5e+4):
        # alphaMax - max modulation frequency
        # df - carrier frequency resolution
        # fMin - min carrier frequency

        # STFT windows' hop
        R = int(np.floor(fs / (2 * alphaMax))) #shift of the stft
        # STFT window length
        Nw = int(fs / df)
        # number of STFT windows
    #     M = int((x.size - Nw) / R + 1)

        # hannind window
        w = np.hanning(Nw)
        # Dirichlet kernel parameter
        P = int(np.round((Nw - 1) / (2 * R)))
        # Dirichlet kernel
        D = np.sum(
            [np.exp(2 * np.pi * 1j * p *(np.arange(Nw) - Nw / 2) / Nw) for p in np.arange(- P, P + 1)], 
            axis=0
        )
        D = D.real
        return fMin, R, Nw, w, D

    def getCS(x, fs):
        fMin, R, Nw, w, D = getParams(fs)

        _, _, X_w = stft(x, fs=fs, window=w, nperseg=Nw, noverlap=Nw - R, nfft=Nw, return_onesided=True)
        f, t, x_w_d = stft(x, fs=fs, window=w * D, nperseg=Nw, noverlap=Nw - R, nfft=Nw, return_onesided=True)

        # here I save some computation time (factor of fMin / fs ~ 4) but later I have to implement the approsimation 
        # to the coherence by neglecting the frequency shift CCoh = CS / CS[0, :]
        X_w = X_w[f >= fMin, :]
        x_w_d = x_w_d[f >= fMin, :]

        CS = np.fft.fft(np.conjugate(X_w) * x_w_d, axis=1).T

        alpha = np.fft.fftfreq(x_w_d.shape[1], R / fs)
        CS = CS[alpha >= 0, :]
        CCoh = CS / CS[0, :]

        alpha = alpha[alpha >= 0]
        f = f[f >= fMin]
        
        EES_CS = np.sum(np.abs(CS), axis=1)
        EES_CCoh = np.sum(np.abs(CCoh), axis=1)

        alpha = alpha[alpha >= 0]
        f = f[f >= fMin]

        return CS, CCoh, f, alpha, EES_CS, EES_CCoh

    df = pd.read_feather(file)
    fs = 1 / df.iloc[1, 0] * 1000
    for iCh in range(1, df.shape[1]):
        newFileCS = file.replace('/', '/' + str(iCh) + '-cs-').replace('.fthr', '.npy')
        newFileCCoh = file.replace('/', '/' + str(iCh) + '-ccoh-').replace('.fthr', '.npy')
        newFileEES_CS = file.replace('/', '/' + str(iCh) + '-EEScs-').replace('.fthr', '.npy')
        newFileEES_CCoh = file.replace('/', '/' + str(iCh) + '-EESccoh-').replace('.fthr', '.npy')
        newFileAlpha = file.split('_16')[0] + '_alpha.npy'
        newFileFreq = file.split('_16')[0] + '_freq.npy'
        if isfile(newFileEES_CS) and isfile(newFileEES_CCoh):
            continue
        CS, CCoh, freq, alpha, EES_CS, EES_CCoh = getCS(df.iloc[:, iCh].to_numpy(), fs)
        
        if not isfile(newFileAlpha):
            np.save(newFileAlpha, alpha)
        if not isfile(newFileFreq):
            np.save(newFileFreq, freq)
#         np.save(newFileCS, CS)
#         np.save(newFileCCoh, CCoh)
#         np.save(newFileCS, CS)
#         np.save(newFileCCoh, CCoh)
        np.save(newFileEES_CS, EES_CS)
        np.save(newFileEES_CCoh, EES_CCoh)

In [2]:
import ipyparallel as ipp
from glob import glob
from time import sleep


with ipp.Cluster() as rc:
    view = rc.load_balanced_view()
    files = sorted(glob("data/*.fthr"))
    asyncresult = view.map_async(calc, files)
    asyncresult.wait_interactive()
    asyncresult.get()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

calc:   0%|          | 0/11471 [00:00<?, ?tasks/s]

Stopping engine(s): 1643364445
engine set stopped 1643364445: {'engines': {'0': {'exit_code': 0, 'pid': 18107, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 18108, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 18109, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 18110, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 18111, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 18112, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 18113, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 18114, 'identifier': '7'}}, 'exit_code': 0}
Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 18095, 'identifier': 'ipcontroller-1643364444-thpw-18090'}


### Visualization

In [13]:
files = glob("data/*.npy")
alphas = sorted([l.split('/')[-1] for l in files if 'alpha.npy' in l])
freqs = sorted([l.split('/')[-1] for l in files if 'freq.npy' in l])
labels = set([l.split('_16')[0].split('/')[-1] for l in files])
EESccohS = [l.split('_16')[0].split('/')[-1] for l in labels if 'EESccoh' in l]
EEScsS = [l.split('_16')[0].split('/')[-1] for l in labels if 'EEScs' in l]
EESccohS = sorted(EESccohS, key=lambda x: (x.split('-')[-2], x.split('-')[-1], x.split('-')[0]))
EEScsS = sorted(EEScsS, key=lambda x: (x.split('-')[-2], x.split('-')[-1], x.split('-')[0]))

display(alphas)
display(freqs)
display(EESccohS)
display(EEScsS)

['29.5_ANALOGMAX-0_alpha.npy',
 '29.5_ANALOGMAX-1_alpha.npy',
 'T7_ANALOGMAX-0_alpha.npy',
 'T7_ANALOGMAX-1_alpha.npy',
 'TRF01_IW976-0032_alpha.npy',
 'TRF10_IW976-0032_alpha.npy']

['29.5_ANALOGMAX-0_freq.npy',
 '29.5_ANALOGMAX-1_freq.npy',
 'T7_ANALOGMAX-0_freq.npy',
 'T7_ANALOGMAX-1_freq.npy',
 'TRF01_IW976-0032_freq.npy',
 'TRF10_IW976-0032_freq.npy']

['1-EESccoh-29.5_ANALOGMAX-0',
 '1-EESccoh-29.5_ANALOGMAX-1',
 '1-EESccoh-T7_ANALOGMAX-0',
 '1-EESccoh-T7_ANALOGMAX-1',
 '1-EESccoh-TRF01_IW976-0032',
 '2-EESccoh-TRF01_IW976-0032',
 '3-EESccoh-TRF01_IW976-0032',
 '4-EESccoh-TRF01_IW976-0032',
 '1-EESccoh-TRF10_IW976-0032',
 '2-EESccoh-TRF10_IW976-0032',
 '3-EESccoh-TRF10_IW976-0032',
 '4-EESccoh-TRF10_IW976-0032']

['1-EEScs-29.5_ANALOGMAX-0',
 '1-EEScs-29.5_ANALOGMAX-1',
 '1-EEScs-T7_ANALOGMAX-0',
 '1-EEScs-T7_ANALOGMAX-1',
 '1-EEScs-TRF01_IW976-0032',
 '2-EEScs-TRF01_IW976-0032',
 '3-EEScs-TRF01_IW976-0032',
 '4-EEScs-TRF01_IW976-0032',
 '1-EEScs-TRF10_IW976-0032',
 '2-EEScs-TRF10_IW976-0032',
 '3-EEScs-TRF10_IW976-0032',
 '4-EEScs-TRF10_IW976-0032']

In [14]:
from datetime import datetime

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

def plotSpectro(sensor, alpha, fileList):
    fileSensor = sorted([file for file in fileList if sensor in file])
    for fileType in fileSensor:
        print(fileType)
        files = sorted(glob('data/' + fileType + '*'))
        EESsNP = np.array([np.load(file) for file in files])
        time = [datetime.fromtimestamp(int(file.split('_')[-1].split('.')[0])) for file in files]
        alphaVec = np.load('data/' + alpha)
        vmax = np.percentile(EESsNP.T, 99.7)
        
        fig = px.imshow(
            EESsNP.T,
            labels=dict(y="Modulation Frequency [Hz]"),            
            x=time,
            y=alphaVec,
            title=fileType,
            zmax=vmax
               )
        
        fig.update_layout(
            yaxis = dict(
                tickmode = 'linear',
                dtick = 50,
                autorange=True,
            )
        )

        fig.write_html('visualization/'+ fileType + ".html")

for alpha in alphas:
    sensor = alpha.split('_alph')[0]
    plotSpectro(sensor, alpha, EESccohS)
    plotSpectro(sensor, alpha, EEScsS)

1-EESccoh-29.5_ANALOGMAX-0
1-EEScs-29.5_ANALOGMAX-0
1-EESccoh-29.5_ANALOGMAX-1
1-EEScs-29.5_ANALOGMAX-1
1-EESccoh-T7_ANALOGMAX-0
1-EEScs-T7_ANALOGMAX-0
1-EESccoh-T7_ANALOGMAX-1
1-EEScs-T7_ANALOGMAX-1
1-EESccoh-TRF01_IW976-0032
2-EESccoh-TRF01_IW976-0032
3-EESccoh-TRF01_IW976-0032
4-EESccoh-TRF01_IW976-0032
1-EEScs-TRF01_IW976-0032
2-EEScs-TRF01_IW976-0032
3-EEScs-TRF01_IW976-0032
4-EEScs-TRF01_IW976-0032
1-EESccoh-TRF10_IW976-0032
2-EESccoh-TRF10_IW976-0032
3-EESccoh-TRF10_IW976-0032
4-EESccoh-TRF10_IW976-0032
1-EEScs-TRF10_IW976-0032
2-EEScs-TRF10_IW976-0032
3-EEScs-TRF10_IW976-0032
4-EEScs-TRF10_IW976-0032
